# Voyages API Example

## Run this example in [Colab](https://colab.research.google.com/github/SignalOceanSdk/SignalSDK/blob/master/docs/examples/jupyter/VoyagesAPI/VoyagesAPI.ipynb). 

## Setup
Install the Signal Ocean SDK:
```
pip install signal-ocean
```
Set your subscription key acquired here: https://apis.signalocean.com/profile

In [ ]:
!pip install signal-ocean

In [ ]:
signal_ocean_api_key = '' #replace with your subscription key

## Call the Voyages API
The Voyages API retrieves information about vessel voyages.

In [ ]:
from signal_ocean import Connection
from signal_ocean.voyages import VoyagesAPI
from signal_ocean.voyages import Vessel, VesselFilter
from signal_ocean.voyages import VesselType, VesselTypeFilter
from signal_ocean.voyages import VesselClass, VesselClassFilter
import pandas as pd
from datetime import date, timedelta

In [ ]:
connection = Connection(signal_ocean_api_key)
api = VoyagesAPI(connection)

### Get voyages for vessel

In [ ]:
# get the imo by using the vessel name
vessel = api.get_imos(VesselFilter('Seaking'))[0]
vessel

In [ ]:
vessel_voyages = api.get_voyages(imo=vessel.imo)

In [ ]:
vessel_voyages_df = pd.DataFrame([v.__dict__ for v in vessel_voyages])
vessel_voyages_df.tail(5)

In [ ]:
vessel_events_df = pd.DataFrame(e.__dict__ for voyage_events in vessel_voyages_df['events'] for e in voyage_events)
vessel_events_df.tail(5)

In [ ]:
vessel_event_details_df = pd.DataFrame(e.__dict__ for event_details in vessel_events_df['event_details'] for e in event_details or [])
vessel_event_details_df.tail(5)

### Get voyages for vessel class

In [ ]:
#get vessel class id for vlcc
vc = api.get_vessel_classes(VesselClassFilter('vlcc'))[0]
vlcc_id = vc.vessel_class_id
vlcc_id

In [ ]:
date_from = date.today() - timedelta(days=30)
recent_vlcc_voyages = api.get_voyages(vessel_class_id=vlcc_id, date_from=date_from)

In [ ]:
recent_vlcc_voyages = pd.DataFrame([v.__dict__ for v in recent_vlcc_voyages])
recent_vlcc_voyages.tail(5)

### Get voyages for vessel in flat format

In [ ]:
# get the imo by using the vessel name
vessel = api.get_imos(VesselFilter('Seaking'))[0]
vessel

In [ ]:
vessel_voyages_flat = api.get_voyages_flat(imo=vessel.imo)

In [ ]:
vessel_voyages_df = pd.DataFrame(v.__dict__ for v in vessel_voyages_flat.voyages)
vessel_voyages_df.tail(5)

In [ ]:
vessel_events_df = pd.DataFrame(v.__dict__ for v in vessel_voyages_flat.events)
vessel_events_df.tail(5)

In [ ]:
vessel_event_details_df = pd.DataFrame(v.__dict__ for v in vessel_voyages_flat.event_details)
vessel_event_details_df.tail(5)

In [ ]:
vessel_voyages_geos_df = pd.DataFrame(v.__dict__ for v in vessel_voyages_flat.geos)
vessel_voyages_geos_df.tail(5)

### Get voyages for vessel class in the flat format

In [ ]:
#get vessel class id for vlcc
vc = api.get_vessel_classes(VesselClassFilter('vlcc'))[0]
vlcc_id = vc.vessel_class_id

date_from = date.today() - timedelta(days=30)
recent_vlcc_voyages_flat = api.get_voyages_flat(vessel_class_id=vlcc_id, date_from=date_from)

In [ ]:
print('Voyages:', len(recent_vlcc_voyages_flat.voyages))
print('Events:', len(recent_vlcc_voyages_flat.events))
print('EventDetails:', len(recent_vlcc_voyages_flat.event_details))
print('Geos:', len(recent_vlcc_voyages_flat.geos))

### Get voyages for vessel incrementally 

Initial requests retrieves voyages mathcing the query criteria and a query token that can be used in the subsequent request.

In [ ]:
# get the imo by using the vessel name
vessel = api.get_imos(VesselFilter('XIN YONG YANG'))[0]
vessel

In [ ]:
vessel_voyages, next_request_token = api.get_incremental_voyages(imo=vessel.imo)
len(vessel_voyages)

Provided the token retrieved for the previous request, the subsequent request retrieves only voyages that have been updated since the previous request. 

In [ ]:
incremental_voyages, next_request_token = api.get_incremental_voyages(imo=9416642, incremental_token=next_request_token)
len(incremental_voyages)

The retrieved information is used to replace any modify voyages from the dataset. Note that returned voyages marked as deleted are only used to filter out voyages. 

In [ ]:
updated_voyage_ids = set(v.id for v in incremental_voyages)
vessel_voyages = [v for v in vessel_voyages if v.id not in updated_voyage_ids]
new_vessel_voyages = [v for v in incremental_voyages if not v.deleted]
vessel_voyages = sorted(vessel_voyages + new_vessel_voyages, key= lambda v: v.voyage_number)
len(vessel_voyages)

### Get voyages for vessel class incrementally 

Voyages for vessel class are retieved and updated in the same way incrementally.

In [ ]:
#get vessel class id for vlcc
vc = api.get_vessel_classes(VesselClassFilter('vlcc'))[0]
vessel_class_id = vc.vessel_class_id
vessel_class_id

In [ ]:
date_from = date.today() - timedelta(days=30)

In [ ]:
voyages, next_request_token = api.get_incremental_voyages(vessel_class_id=vessel_class_id, date_from=date_from)
len(voyages)

In [ ]:
incremental_voyages, next_request_token = api.get_incremental_voyages(vessel_class_id=vessel_class_id, date_from=date_from, incremental_token=next_request_token)
len(incremental_voyages)

In [ ]:
updated_voyage_ids = set(v.id for v in incremental_voyages)
voyages = [v for v in voyages if v.id not in updated_voyage_ids and not v.deleted]
new_voyages = [v for v in incremental_voyages if not v.deleted]
voyages = sorted(voyages + new_voyages, key= lambda v: (v.imo, v.voyage_number))
len(voyages)

### Get voyages for vessel class incrementally in the flat format

Voyages may be retrieved and updated incrementally in the flat format. 

In [ ]:
#get vessel class id for vlcc
vc = api.get_vessel_classes(VesselClassFilter('vlcc'))[0]
vessel_class_id = vc.vessel_class_id
vessel_class_id

In [ ]:
date_from = date.today() - timedelta(days=30)

In [ ]:
voyages_flat, next_request_token = api.get_incremental_voyages_flat(vessel_class_id=vessel_class_id, date_from=date_from)

In [ ]:
voyages = voyages_flat.voyages
events = voyages_flat.events
event_details = voyages_flat.event_details
geos = voyages_flat.geos

In [ ]:
len(voyages), len(events), len(event_details), len(geos)

In [ ]:
incremental_voyages_flat, next_request_token = api.get_incremental_voyages_flat(vessel_class_id=vessel_class_id, date_from=date_from, incremental_token=next_request_token)

In [ ]:
len(incremental_voyages_flat.voyages)

In this case the update step is applied to voyages, events, event details and geos datasets.

In [ ]:
updated_voyage_ids = set(v.id for v in incremental_voyages_flat.voyages)
voyages = [v for v in voyages if v.id not in updated_voyage_ids and not v.deleted]
new_voyages = [v for v in incremental_voyages_flat.voyages if not v.deleted]
voyages = sorted(voyages + new_voyages, key= lambda v: (v.imo, v.voyage_number))
len(voyages)

In [ ]:
updated_event_ids = set(e.id for e in events if e.voyage_id in updated_voyage_ids)
events = [e for e in events if e.id not in updated_event_ids]
events = sorted(events + list(incremental_voyages_flat.events), key= lambda e: e.id)
len(events)

In [ ]:
event_details = [e for e in event_details if e.id not in updated_event_ids]
event_details = sorted(event_details + list(incremental_voyages_flat.event_details), key= lambda e: e.id)
len(event_details)

In [ ]:
updated_geo_ids = set(g.id for g in incremental_voyages_flat.geos)
geos = [g for g in geos if g.id not in updated_geo_ids] + list(incremental_voyages_flat.geos)
len(geos)